In [65]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
import sqlalchemy
from sqlalchemy import create_engine


In [66]:
# read channels, videos, views, subs data from csv files
channels = pd.read_csv('channels.csv')
videos = pd.read_csv('videos.csv')
views = pd.read_csv('views.csv')
subs = pd.read_csv('subs.csv')

In [67]:
channels.rename(columns={'pK':'channel_ID'}, inplace=True)

In [68]:
videos.rename(columns={'pK':'video_ID'}, inplace=True)

In [69]:
videos['duration'] = pd.to_timedelta(videos['duration'])
videos['duration'] = videos['duration'] / pd.Timedelta(minutes=1)

In [70]:
videos

,video_ID,title,channel_ID,category,duration,rating
0,1,2 hours soft rain sounds,1,background,0.0,6.8
1,2,New Tetris?!?!,2,gaming,0.0,8.0
2,3,My apology,2,discussion,0.0,3.4
3,4,Dog TikTok Compilation,3,animals,0.0,8.7
4,5,Not A Rickroll,4,music,0.0,2.6
5,6,Learn MongoDB,5,educational,0.0,5.2
6,7,Learn Python in 30 seconds!,5,educational,0.0,8.9
7,8,Let's Play Tears of the Kingdom,2,gaming,0.0,7.5
8,9,How to Travel for Cheap,6,discussion,0.0,8.5
9,10,Big Announcement...,4,discussion,0.0,9.0


In [71]:
#save all dataframes to csv files
channels.to_csv('channels.csv', index=False)
videos.to_csv('videos.csv', index=False)
views.to_csv('views.csv', index=False)
subs.to_csv('subs.csv', index=False)

In [72]:
password = 'dbuserdbuser'
db_name = 'launch_proj'
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [73]:
channels.to_sql(name = 'channels', con=connection, if_exists='replace', index=False)
videos.to_sql(name = 'videos', con=connection, if_exists='replace', index=False)
views.to_sql(name = 'views', con=connection, if_exists='replace', index=False)
subs.to_sql(name = 'subs', con=connection, if_exists='replace', index=False)

20

In [74]:
#test query
pd.read_sql_query('SELECT * FROM subs', connection)

,sub_ID,channel_ID,user_ID,subscription_date,status
0,1,2,1,2022-04-01,subscribed
1,2,2,1,2022-07-01,unsubscribed
2,3,1,2,2022-08-01,subscribed
3,4,4,3,2022-06-01,unsubscribed
4,5,4,2,2022-10-01,subscribed
5,6,5,3,2022-07-01,subscribed
6,7,2,2,2022-05-01,unsubscribed
7,8,1,4,2022-06-01,unsubscribed
8,9,4,4,2022-06-01,unsubscribed
9,10,5,3,2022-07-01,unsubscribed


In [75]:
#merge tables to add them to mongodb database
df = pd.merge(channels, videos, on='channel_ID')
df = pd.merge(df, views, on='video_ID')
df = pd.merge(df, subs, on='channel_ID')

df.to_csv('merged.csv', index=False)

In [76]:
vid_dict = videos.to_dict('records')
chan_dict = channels.to_dict('records')
view_dict = views.to_dict('records')
sub_dict = subs.to_dict('records')

In [77]:
#for each entry in chan_dict, add a new key called 'subs' and add the corresponding values from sub_dict by matching channel_ID
for i in range(len(chan_dict)):
    chan_dict[i]['subs'] = []
    for j in range(len(sub_dict)):
        if chan_dict[i]['channel_ID'] == sub_dict[j]['channel_ID']:
            chan_dict[i]['subs'].append(sub_dict[j])

In [78]:
#for each entry in vid_dict, add a key called views and make its value the corresponding list of view dictionary by video_ID
for i in range(len(vid_dict)):
    vid_dict[i]['views'] = []
    for j in range(len(view_dict)):
        if vid_dict[i]['video_ID'] == view_dict[j]['video_ID']:
            vid_dict[i]['views'].append(view_dict[j])

In [79]:
#for each entry in chan_dict, add a new key called 'videos' and add the corresponding values from vid_dict by matching channel_ID
for i in range(len(chan_dict)):
    chan_dict[i]['vids'] = []
    for j in range(len(vid_dict)):
        if chan_dict[i]['channel_ID'] == vid_dict[j]['channel_ID']:
            chan_dict[i]['vids'].append(vid_dict[j])

chan_dict

[{'channel_ID': 1,
  'name': 'Background Music Channel',
  'description': 'a variety of different white noise and background music',
  'owner_ID': 1,
  'subscribers': 30000,
  'subs': [{'sub_ID': 3,
    'channel_ID': 1,
    'user_ID': 2,
    'subscription_date': '2022-08-01',
    'status': 'subscribed'},
   {'sub_ID': 8,
    'channel_ID': 1,
    'user_ID': 4,
    'subscription_date': '2022-06-01',
    'status': 'unsubscribed'}],
  'vids': [{'video_ID': 1,
    'title': '2 hours soft rain sounds',
    'channel_ID': 1,
    'category': 'background',
    'duration': 0.0,
    'rating': 6.8,
    'views': [{'view_ID': 4,
      'video_ID': 1,
      'user_ID': 2,
      'view_date': '2022-08-30',
      'duration_watched': 120.0,
      'feedback': 7.0},
     {'view_ID': 8,
      'video_ID': 1,
      'user_ID': 3,
      'view_date': '2022-05-05',
      'duration_watched': 20.0,
      'feedback': 10.0}]}]},
 {'channel_ID': 2,
  'name': 'G4mer',
  'description': 'I play video games and make commentar

In [82]:
final = chan_dict

In [83]:
#add the final dictionary to mongodb
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['launch_proj']
collection = db['channels']
collection.insert_many(final)